In [1]:
# Importando las librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importando el conjunto de datos
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
# Codificación de datos categóricos
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])  # Codificando la columna 'Geography'
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])  # Codificando la columna 'Gender'



In [4]:
# Aplicando OneHotEncoder para la columna 'Geography'
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'),[1])], remainder='passthrough')
# hago drop = 'first' para evitar el problema de las variables ficticias
# No lo hago para la columna género porque ya tengo 0 0 1 según sea mujer u hombre.
# Si lo hubiese aplicado -> OneHotEncoder(drop='first'),[1,2])
X = np.array(ct.fit_transform(X))

In [5]:
# Dividiendo el conjunto de datos en conjunto de entrenamiento y conjunto de prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [6]:
# Escalado de características (Feature Scaling)
# Fundamental en Redes Neuronales
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## PARTE 2- CONSTRUIR LA RNA (Red Neuronal Artificial)

# Importar Keras y librerías adicionales

In [7]:
from tensorflow import keras
from tensorflow.keras.models import Sequential #Para inicializar los parámetros de la Red Neuronal
from tensorflow.keras.layers import Dense # Para declarar y crear las capas intermedias de la Red Neuronal

## PARTE 3- CONSTRUIR LA RNA

In [8]:
# Inicializar la RNA
classifier = Sequential()


In [9]:
# Añadir capas de entrada y primeras capa oculta
# Paso 1 - Inicializar los pesos aleatoriamente con valores cercanos a 0 (pero no 0)
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = "relu", input_dim = 11 ))
# Buena regla para los units (nodos) -> Media entre los nodos de entrada y los nodos de salida ( (11 + 1)/2 = 6)

In [10]:
# Añadir la siguiente capa oculta
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = "relu"))
# La siguiente capa oculta ya sabe lo que espera de la primera capa -> input_dim no es necesario
# podemos dejar el mismo número de nodos en la siguiente capa oculta. Tb la misma función


In [11]:
# Añadir la capa de salida
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = "sigmoid"))
# en la capa de salida quiero una probabilidad -> función de activació: función sigmoide

In [12]:
# Compilar la RNA
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=["accuracy"])

In [14]:
# Ajustamos la RNA al conjunto de entrenamiento (Paso 2)
classifier.fit(X_train,y_train, batch_size=10, epochs = 100)
#batch_size=10 -> procesar 10 elementos -> Corregir los pesos -> procesar 10 elementos -> Corregir los pesos (más óptimo)
#epochs=100 -> Dará 100 pasadas a nuestros datos

Epoch 1/100


800/800 [==============================] - 2s 1ms/step - loss: 0.4794 - accuracy: 0.7959
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4236 - accuracy: 0.8008
Epoch 3/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4162 - accuracy: 0.8300
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4112 - accuracy: 0.8306
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4080 - accuracy: 0.8330
Epoch 6/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4049 - accuracy: 0.8346
Epoch 7/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4035 - accuracy: 0.8341
Epoch 8/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4020 - accuracy: 0.8338
Epoch 9/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4012 - accuracy: 0.8355
Epoch 10/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4002 - accu

# Parte 3 - Evaluar el modelo y calcular predicciones  finales

In [19]:
# Predicción de los resultados con el conjunto de Testing
y_pred = classifier.predict(X_test)
y_pred = (y_pred>0.5)

63/63 [==============================] - 0s 1ms/step


In [20]:
# Elaborar una matriz de confusión
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [21]:
cm

array([[1514,   81],
       [ 203,  202]], dtype=int64)